In [7]:
# to capture game screen
from mss import mss
# to provide keystroke inputs to the game
import pydirectinput
import numpy as np
# to perform OCR on game screen
import time
# to create custom gym environment
from gymnasium import Env
# to define action and observation space
from gymnasium.spaces import Discrete, Box


# 0: 'bird'
# 1: 'cactus'
# 2: 'restart'
# 3: 't-rex'

In [78]:
# code to detect the objects in image
from ultralytics import YOLO
import cv2

yolo_model = YOLO("yolo_model/object_identification_model.pt")


def getObjectPositions(observation):
    observation = cv2.cvtColor(observation, cv2.COLOR_GRAY2RGB)
    results = yolo_model.predict(source=observation, conf=0.25, imgsz=640, verbose=False)
    result = results[0]
    boxes = result.boxes

    if len(boxes) == 0:
        return boxes.data

    # Group boxes by class and find leftmost for each class, that have x1>106
    class_leftmost = {}

    for i, box in enumerate(boxes):
        class_id = int(box.cls)
        x1 = float(box.xyxy[0][0])  # leftmost x coordinate

        # Keep the box with smallest x1 (leftmost) for each class
        if class_id in [0, 1]:
            if class_id not in class_leftmost or x1 < class_leftmost[class_id]['x1']:
                class_leftmost[class_id] = {'index': i, 'x1': x1}

    # Extract the indices of leftmost boxes for each class
    leftmost_indices = [info['index'] for info in class_leftmost.values()]

    # Return only the leftmost boxes data
    filtered_data = boxes.data[leftmost_indices]
    return filtered_data


In [79]:
def format_onservation(filtered_data):
    obs = np.array([-1,-1,0,0], dtype=np.float32)
    # {0: 'bird', 1: 'cactus', 2: 'restart', 3: 't-rex'}
    #cactus-x1, bird-x1, bird-y1, speed
    for box in filtered_data:
        if int(box[5]) == 1:  # cactus
            obs[0] = box[0]  # x1
            obs[3] = box[0]  # speed
        elif int(box[5]) == 0:  # bird
            obs[1] = box[0]  # x1
            obs[2] = box[1]  # y1
    return obs

In [97]:
LEFT_THRESHOLD = 130
MIN_SPEED = 250
MIN_TIME = 0.025

class WebGame(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(
            low=np.array([
                -1,-1, 0, 0  #cactus-x1, bird-x1, bird-y1, speed
            ], dtype=np.float32),
            high=np.array([
                640, 640, 256, 1000
            ], dtype=np.float32),
            dtype=np.float32
        )

        self.action_space = Discrete(3)

        # define the extraction parameters for the game
        self.cap = mss()
        monitor_2 = self.cap.monitors[1]
        self.game_location = {
            'top': monitor_2["top"] + 150,
            'left': monitor_2["left"] + 400,
            'width': 640,
            'height': 256
        }

        #variables to calculate speed
        self.pcx = 641
        self.speed = 350
        self.last_speed_calculation_time = -1

    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))
        raw_rgb = raw[:, :, :3].astype(np.uint8)
        gray = cv2.cvtColor(raw_rgb, cv2.COLOR_RGB2GRAY)
        filtered_data = getObjectPositions(gray)
        formated_observation = format_onservation(filtered_data)
        formated_observation[3] = self.calculateSpeed(formated_observation[0])
        # print(formated_observation[3],end='|')
        # print(formated_observation)
        return formated_observation

    def step(self, action):
        action_map = {
            0: 'space',
            1: 'down',
            2: 'no_op',
        }
        if action != 2:
            pydirectinput.press(action_map[action])

        observation = self.get_observation()
        # Get object positions and names
        try:
            reward = self.calculate_reward(observation)
        except Exception as e:
            print("Error in object detection or reward calculation:", e)
            reward = -10

        info = {}
        return observation, reward, bool(observation[3] <= MIN_SPEED), False, info

    def reset(self, seed=None, options=None):
        print(f"\n🔄 Episode reset")
        # Reset game state
        time.sleep(0.25)
        monitor_2 = self.cap.monitors[1]
        pydirectinput.click(x=monitor_2["left"] + 300, y=monitor_2["top"] + 200)
        time.sleep(0.25)
        pydirectinput.press('space')
        time.sleep(0.25)
        pydirectinput.press('space')
        self.pcx = 641
        self.speed = 350
        self.last_speed_calculation_time = -1
        # print("waiting 3 sec")
        time.sleep(3)
        # print("3 sec over, getting observation")
        obs = self.get_observation()
        while obs[0]==-1:
            # print("waiting", end=' ')
            time.sleep(0.25)
            obs = self.get_observation()
        return obs, {}

    def calculateSpeed(self, cx):
        if cx == -1:
            # print("cx==-1, returning last speed:", self.speed)
            return self.speed
        if self.pcx==641 or self.last_speed_calculation_time==-1:
            self.pcx = cx
            self.last_speed_calculation_time = time.time()
            # print("pcx initialized, returning speed:", self.speed)
            return self.speed
        else:
            current_time = time.time()
            time_diff = current_time - self.last_speed_calculation_time
            if time_diff < MIN_TIME:
                # print("time_diff<0.1, returning last speed:", self.speed)
                return self.speed
            distance_moved = self.pcx - cx

            if distance_moved == 0:
                # print("distance_moved==0, returning speed 0")
                return 0

            if distance_moved>0:
                self.speed = distance_moved / time_diff

            self.last_speed_calculation_time = current_time
            self.pcx = cx

            # print("distance_moved:", distance_moved, "time_diff:", time_diff, "calculated speed:", self.speed)

            return self.speed


    def calculate_reward(self, filtered_data):
        reward = 0
        cactus_x1 = filtered_data[0]
        bird_x1 = filtered_data[1]
        speed = filtered_data[3]

        if speed <= MIN_SPEED:
            print("☠️", end='')
            return -10  # Game over

        # Reward for staying alive
        reward += 1

        # if cactus is passed
        if cactus_x1 != -1 and cactus_x1 < LEFT_THRESHOLD:
            print("🌵", end='')
            reward += 5

        # if bird is passed
        if bird_x1 != -1 and bird_x1 < LEFT_THRESHOLD:
            print("🦅", end='')
            reward += 5

        print("", end='|')

        return reward


In [98]:
env = WebGame()
env.get_observation()

array([         -1,          -1,           0,         350], dtype=float32)

In [83]:
for episodes in range(10):
    obs = env.reset(1)
    done = False
    total_reward = 0
    while not done:
        obs, reward, done, truncated, info = env.step(env.action_space.sample())
        total_reward += reward
    print("total reward:", total_reward, ". for episodes:", episodes)


🔄 Episode reset
waiting 3 sec
3 sec over, getting observation
waiting waiting 453.74213||442.06357||318.0667||310.42407||191.36798||183.60199||☠️☠️total reward: -14 . for episodes: 0

🔄 Episode reset
waiting 3 sec
3 sec over, getting observation
waiting waiting 451.36517||438.0572||319.51233||206.12717||196.9386||188.83475||182.2143||174.91191||168.9851||159.91202||☠️☠️total reward: -10 . for episodes: 1

🔄 Episode reset
waiting 3 sec
3 sec over, getting observation
waiting waiting 436.01422||317.40656||203.0961||196.2443||188.88052||181.22757||☠️☠️total reward: -14 . for episodes: 2

🔄 Episode reset
waiting 3 sec
3 sec over, getting observation
waiting waiting 456.02527||341.94324||331.1715||320.3713||200.81708||☠️☠️☠️total reward: -25 . for episodes: 3

🔄 Episode reset
waiting 3 sec
3 sec over, getting observation
waiting waiting 459.39056||343.4803||225.28473||203.62857||☠️☠️total reward: -16 . for episodes: 4

🔄 Episode reset
waiting 3 sec
3 sec over, getting observation
waiting w

In [99]:
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker

In [94]:
env_checker.check_env(env)


🔄 Episode reset

🔄 Episode reset
|
🔄 Episode reset
|☠️
🔄 Episode reset
||||||||

In [95]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self) -> None:
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)


In [ ]:
from stable_baselines3 import DQN
model = DQN('MlpPolicy', env, tensorboard_log=LOG_DIR, verbose=2, buffer_size=10000, learning_starts=1000, learning_rate=0.01)
model.learn(total_timesteps=10000000, callback=callback)

In [102]:
 # Load the trained model
# model = DQN.load('train2/best_model_70000.zip')
model = DQN.load('train/best_model_22000.zip')

# Test the model
episodes = 5
for episode in range(episodes):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    step_count = 0

    print(f"Episode {episode + 1} starting...")

    while not done and step_count < 1000:
        # Use the trained model to predict actions
        action, _ = model.predict(obs, deterministic=False)
        print(action, end=" | ")
        action = int(action)  # Convert numpy array to scalar

        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
        step_count += 1

        # Add small delay to see the gameplay
        time.sleep(0.1)

    print(f"Episode {episode + 1} finished - Total reward: {total_reward}, Steps: {step_count}")



🔄 Episode reset
Episode 1 starting...
1 | |1 | |0 | |1 | |2 | |0 | |1 | ☠️Episode 1 finished - Total reward: -4, Steps: 7

🔄 Episode reset
Episode 2 starting...
2 | |0 | |0 | |2 | |0 | |2 | |1 | |1 | |2 | |1 | ☠️Episode 2 finished - Total reward: -1, Steps: 10

🔄 Episode reset
Episode 3 starting...
1 | |1 | |0 | |1 | ☠️Episode 3 finished - Total reward: -7, Steps: 4

🔄 Episode reset
Episode 4 starting...
2 | |0 | |0 | |2 | |0 | |1 | |1 | |2 | |1 | ☠️Episode 4 finished - Total reward: -2, Steps: 9

🔄 Episode reset
Episode 5 starting...
0 | |0 | |0 | |2 | 🌵|0 | 🌵|0 | ☠️Episode 5 finished - Total reward: 5, Steps: 6
